In [7]:
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE,SCIP_HEURTIMING
import pyscipopt
import sys , os
from tqdm import tqdm
import random
import pandas as pd
import numpy as np

In [8]:
test = pyscipopt.scip.Model()
test.setIntParam("heuristics/rins/freq",-1)
test.readProblem("/Users/oukeikou/Desktop/sunruoyao/DATA/easy-sample/markshare_4_0.mps")

print("目标函数类型:", test.getObjectiveSense())

目标函数类型:original problem has 34 variables (30 bin, 0 int, 0 impl, 4 cont) and 4 constraints
 minimize


In [9]:
class FixedVarsAtNode(Eventhdlr):
    """PySCIPOpt Event handler to write fixed vars of each node to a text file."""

    def __init__(self,random_seed):

        self.count=0
        self.count_limit=175001

        self.batch_size = 5000  # 设置每个文件记录的节点数
        self.batch_number = 0  # 当前文件的批次号

        # 数据文件夹路径
        self.data_folder = "/Users/oukeikou/Desktop/sunruoyao/marks_output/origin"
        self.original_obj_values = self.read_data_file()

        #make random seed
        self.random_seed = random_seed
        self.random = random.Random(random_seed)

        self.nextnode = False

        # 设置每累积1000条数据就输出一个文件
        self.df_var_info = pd.DataFrame(columns=['Var','Depth','Type','LowerBound','UpperBound','GLB','GUB','y_label'])
        self.data_threshold = 5000

    
    def read_data_file(self):
        obj_values_numpy = np.empty((self.batch_size,))
        data = pd.DataFrame(columns=['Node', 'ObjVal'])
        file_name = os.path.join(self.data_folder, f"marks_output_{self.batch_number}_y_info.csv")
        if os.path.exists(file_name):
            data = pd.read_csv(file_name)
            obj_values_numpy = data['ObjVal'].values
            obj_values_numpy = np.insert(obj_values_numpy, 0, 0)
        return obj_values_numpy
        

    def eventexec(self, event):
        if self.count<self.count_limit:

            if self.nextnode:
                #比较是否有改变 目标函数类型: minimize
                obj_value_heuristic = float(self.model.getObjVal())
                self.depth = self.model.getDepth()
                
                if obj_value_heuristic < float(self.original_obj_values[self.count % self.batch_size]):
                    #写入node信息
                    
                    self.transvars = self.model.getVars(transformed=True)
                    for var in self.transvars:
                        var_name = var.name
                        Glb = var.getUbGlobal()#获取变量的全局上界
                        Gub = var.getLbGlobal()#获取变量的全局下界
                        lb = var.getLbLocal()  # 获取变量的局部下界
                        ub = var.getUbLocal()  # 获取变量的局部上界
                        var_type = var.vtype()
                    
                        var_info_add_df ={'Var': var_name, 'Depth':self.depth,'Type': var_type, 'LowerBound': lb, 'UpperBound': ub, 'GLB':Glb, 'GUB':Gub ,'y_label':1}
                        #append to df_var_info
                        self.df_var_info = self.df_var_info.append(var_info_add_df, ignore_index=True)
                    if self.count % self.data_threshold == 0:
                        self.save_data()
                    
                #保存数据
                else:
                    #写入node信息
                    self.transvars = self.model.getVars(transformed=True)
                    for var in self.transvars:
                        var_name = var.name
                        Glb = var.getUbGlobal()#获取变量的全局上界
                        Gub = var.getLbGlobal()#获取变量的全局下界
                        lb = var.getLbLocal()  # 获取变量的局部下界
                        ub = var.getUbLocal()  # 获取变量的局部上界
                        var_type = var.vtype()
                    
                        var_info_add_df ={'Var': var_name,'Depth':self.depth, 'Type': var_type, 'LowerBound': lb, 'UpperBound': ub, 'GLB':Glb, 'GUB':Gub ,'y_label':0}
                        #append to df_var_info
                        self.df_var_info = self.df_var_info.append(var_info_add_df, ignore_index=True)
                        # with open('/Users/oukeikou/Desktop/sunruoyao/marks_output/re.txt', "a") as file:
                        #     file.write(f"{obj_value_heuristic}, {float(self.original_obj_values[self.count % self.batch_size])}\n")
                    if self.count % self.data_threshold == 0:
                        self.save_data()

            else:
                pass

            use_heuristic = self.random.choice([True, False])
            if use_heuristic: #use H in the next node
                self.model.setIntParam("heuristics/rins/freq",1)
                self.nextnode = True
            else:
                self.model.setIntParam("heuristics/rins/freq",100)
                self.nextnode = False

            if self.count % self.batch_size == 0:
                # 切换到下一批次的数据文件
                self.batch_number += 1
                self.original_obj_values = self.read_data_file()

            self.count += 1
        else:
            pass
         
      
    def save_data(self):
        #输出为csv文件
        file_name = 'marks_output/withH/batch_'+str(self.batch_number) +"_info.csv"
        self.df_var_info.to_csv(file_name, index=False)
        self.df_var_info = pd.DataFrame(columns=['Var','Depth','Type','LowerBound','UpperBound','GLB','GUB','y_label'])
        

    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.NODESOLVED, self)

    def eventexit(self):
        self.model.dropEvent(SCIP_EVENTTYPE.NODESOLVED, self)


In [ ]:
if __name__ == "__main__":
    # Specify the output file path
    #output_var_info_file = "var_info.txt"
    
    test = pyscipopt.scip.Model()
    # test.setIntParam("heuristics/rins/freq",100)
    test.readProblem("/Users/oukeikou/Desktop/sunruoyao/DATA/easy-sample/markshare_4_0.mps")
    print("read done")


    # Create and add event handler with the specified output file
    eventhdlr = FixedVarsAtNode(10)
    test.includeEventhdlr(eventhdlr, "FixedVarsAtNode", "Python event handler to write fixed variables after each solved node")

    # Optimize the problem
    test.optimize()
    print("optimized")
    
    # eventhdlr.write_2_file('markshare_4_0')

read doneoriginal problem has 34 variables (30 bin, 0 int, 0 impl, 4 cont) and 4 constraints

presolving:
(round 1, fast)       4 del vars, 0 del conss, 4 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       4 del vars, 4 del conss, 4 add conss, 4 chg bounds, 4 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 3, exhaustive) 4 del vars, 4 del conss, 4 add conss, 4 chg bounds, 4 chg sides, 0 chg coeffs, 4 upgd conss, 0 impls, 0 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int -, cont +), (fixed: bin -, int +, cont -)
   (0.0s) no symmetry present
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 4 deleted vars, 4 deleted constraints, 4 added constraints, 4 tightened bounds, 0 added holes, 4 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3